In [143]:
import pandas as pd
import json
import datetime

In [144]:
path = "/Users/hogni/Library/Mobile Documents/com~apple~CloudDocs/Bachelor Project/Videos/Test/Data/"

columns = ["frame_id", "timestamp", "counter_area", "ObjectClass", "UniqueID", "bearing", "CountingDirection", "Angle"]

counter_df = pd.read_csv(f"{path}counterData-2021-03-10-60488f6516752f0054c6f771.csv", names=columns)

In [145]:
counter_df = counter_df[counter_df.duplicated(subset=["UniqueID"], keep=False)].sort_values("UniqueID")

In [146]:
counter_df['timestamp'] = pd.to_datetime(counter_df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%fZ')

In [147]:
counter_df

,frame_id,timestamp,counter_area,ObjectClass,UniqueID,bearing,CountingDirection,Angle
0,181,2021-03-10 09:20:58.348,start,bicycle,1454,92.161079,leftright_topbottom,29.842766
1,182,2021-03-10 09:20:58.471,end,bicycle,1454,93.366461,leftright_topbottom,36.530463
2,208,2021-03-10 09:21:01.498,end,bicycle,1458,92.726311,leftright_topbottom,35.890313
3,208,2021-03-10 09:21:01.498,start,bicycle,1458,92.726311,leftright_topbottom,30.407997
4,216,2021-03-10 09:21:02.407,start,bicycle,1459,92.436648,leftright_topbottom,30.118335
5,218,2021-03-10 09:21:02.654,end,bicycle,1459,93.576334,leftright_topbottom,36.740337
14,637,2021-03-10 09:21:51.264,start,bicycle,1471,289.653824,rightleft_bottomtop,47.335511
13,630,2021-03-10 09:21:50.458,end,bicycle,1471,273.366461,rightleft_bottomtop,36.530463
11,625,2021-03-10 09:21:49.897,end,bicycle,1472,279.462322,rightleft_bottomtop,42.626325
12,628,2021-03-10 09:21:50.233,start,bicycle,1472,272.726311,rightleft_bottomtop,30.407997


In [25]:
def calculate_true_heading(frame_orientation, bearing):
    true_heading = bearing + frame_orientation
    if true_heading > 360:
        true_heading = true_heading - 360
    return true_heading

In [26]:
calculate_true_heading(200, 270)

110

In [27]:
def calculate_meters_per_second(distance_meters, time_seconds):
    return distance_meters/time_seconds

In [28]:
def desire_path_predictor(UniqueID, true_heading, speed):
    pass